In [1]:
import pandas as pd
import os

project_folder="C:/Users/hanson/OneDrive/Grad/Grad Project/data_realtorCA/"
output_folder=project_folder+"output"

df=pd.read_csv(output_folder+"/"+"all_after_preprocessing"+'.csv')

## Check column "parkingType"

In [2]:
pd.set_option("display.max_rows", 5, "display.max_columns", None) 
#pd.set_option('display.max_colwidth', None) #description full display


In [3]:
df["parkingType"].value_counts()

Attached Garage                              2090
Underground                                  1874
                                             ... 
Attached Garage, Carport, Visitor Parking       1
Oversize, Gravel, RV                            1
Name: parkingType, Length: 131, dtype: int64

In [4]:
df[df["parkingType"]=="Garage"].typeBuilding.value_counts()

House              762
Row / Townhouse    250
                  ... 
Triplex              2
Vacant Land          1
Name: typeBuilding, Length: 6, dtype: int64

## convert multiple parkingType tags into columns with true/false value

In [5]:
# split each of the strings into a list
df_3=df.copy()
df_3['parkingType'] = df_3['parkingType'].str.replace(" ","")
df_3['parkingType'] = df_3['parkingType'].str.replace(" ","")

df_3['parkingType'] = df_3['parkingType'].str.split(pat=',')

# collect all unique tags from those lists
tags = set(df_3['parkingType'].explode().values)
print(tags)

{'Other', 'AttachedGarage', 'NoGarage', 'Underground', 'NotListed', 'Open', 'None', 'InsideEntry', 'Surfaced', 'Oversize', 'RV', 'Tandem', 'Gravel', 'Garage', 'Shared', 'DetachedGarage', 'Carport', 'StreetPermit', 'Covered', 'Interlocked', 'VisitorParking'}


In [6]:
# create a new Boolean column for each tag
for tag in tags:
    df_3[tag] = [tag in df_3['parkingType'].loc[i] for i in df_3.index]

#display(df_3)

In [7]:
parkingCol=['Shared', 'DetachedGarage', 'AttachedGarage', 'Garage', 'Covered', 'StreetPermit', 'Other', 'NoGarage', 'NotListed', 'Open', 'Oversize', 'Gravel', 'VisitorParking', 'Tandem', 'Interlocked', 'Surfaced', 'RV', 'None', 'Underground', 'InsideEntry', 'Carport']

## Check which tag parkingType are most popular, show it from the most to least

In [8]:
newdict={}
for col in parkingCol:
    newdict[col]=df_3[col][df_3[col]==True].count()
    #print(col+" "+str(df_3[col][df_3[col]==True].count()))
sorted_keys = sorted(newdict, key=newdict.get, reverse=True)  # [1, 3, 2]
sorted_dict = {}
for w in sorted_keys:
    sorted_dict[w] = newdict[w]

print(sorted_dict) # {1: 1, 3: 4, 2: 9}

{'Underground': 3501, 'AttachedGarage': 2501, 'VisitorParking': 1967, 'NotListed': 1656, 'Garage': 1107, 'DetachedGarage': 747, 'Surfaced': 318, 'InsideEntry': 253, 'NoGarage': 207, 'Carport': 108, 'Open': 83, 'None': 82, 'Gravel': 81, 'Interlocked': 47, 'Shared': 33, 'Oversize': 33, 'Tandem': 22, 'Covered': 7, 'Other': 6, 'StreetPermit': 4, 'RV': 4}


## Drop columns that are ambiguous
## drop "NoGarage", "None", "NotListed" because it's already true/false, "Other", "StreetPermit" because it's useless
## drop "Tandem" because it is bike parking, 
## Ambiguous terms like "Covered" is merged into "Carport"
## "RV" is merged with "Oversize"
## “Open” is merged with “Surfaced”. 

In [9]:
parking_all_tags=list(sorted_dict)
df_3.loc[(df_3['Carport']==True)|(df_3['Covered']==True), 'Carport'] = True
df_3.loc[(df_3['Oversize']==True)|(df_3['RV']==True), 'Oversize'] = True
df_3.loc[(df_3['Surfaced']==True)|(df_3['Open']==True), 'Surfaced'] = True

df_3 = df_3.drop(["NoGarage", "None", "NotListed", "Other", "StreetPermit", "Tandem", "Covered", "RV","Open"], axis=1)

In [10]:
for x in ["NoGarage", "None", "NotListed", "Other", "StreetPermit", "Tandem", "Covered", "RV", "Open"]:
    print(x)
    parkingCol.remove(x)
    
newdict={}
for col in parkingCol:
    newdict[col]=df_3[col][df_3[col]==True].count()
    #print(col+" "+str(df_3[col][df_3[col]==True].count()))
sorted_keys = sorted(newdict, key=newdict.get, reverse=True)  # [1, 3, 2]
sorted_dict = {}
for w in sorted_keys:
    sorted_dict[w] = newdict[w]

print(sorted_dict) # {1: 1, 3: 4, 2: 9}

NoGarage
None
NotListed
Other
StreetPermit
Tandem
Covered
RV
Open
{'Underground': 3501, 'AttachedGarage': 2501, 'VisitorParking': 1967, 'Garage': 1107, 'DetachedGarage': 747, 'Surfaced': 365, 'InsideEntry': 253, 'Carport': 115, 'Gravel': 81, 'Interlocked': 47, 'Oversize': 35, 'Shared': 33}


In [11]:
df_3.head()

,MLS,price,city,communityName,address,postal,description,typeBuilding,title,bedroomAboveGrade,bedroomBelowGrade,bathroomTotal,bathroomPartial,parkingType,totalParkingSpaces,landSize,storeys,basementType,style,buildingAmenities,maintenanceFees,maintenanceManagementCompany,cooling,heatingType,exteriorFinish,buildingFeatures,amenitiesNearby,AttachedGarage,Underground,InsideEntry,Surfaced,Oversize,Gravel,Garage,Shared,DetachedGarage,Carport,Interlocked,VisitorParking
0,W5238356,958800,Brampton,Sandringham-Wellington,232 BRUSSELS AVEBrampton,L6Z0G1,"Beautiful, Bright & Spacious Home In The Heart...",House,Freehold,4,1,4,0,[Garage],4,22.47 x 99.25 FT,2,N/A (Finished),Semi-detached,NotListed,NotListed,NotListed,Central air conditioning,Forced air (Natural gas),Brick,NotListed,"Hospital, Park, Public Transit, Schools",False,False,False,False,False,False,True,False,False,False,False,False
1,W5226917,970000,Brampton,Downtown Brampton,#LOT 12 -57 MCMURCHY AVE SBrampton,L6Y1Y6,This Is An Assignment Sale !!!Sunfield Communi...,Row / Townhouse,Condominium,3,0,4,0,[AttachedGarage],1,NotListed,3,N/A (Unfinished),NotListed,NotListed,NotListed,Not Decided Yet,Central air conditioning,Forced air (Natural gas),Brick,NotListed,"Hospital, Park, Public Transit, Schools",True,False,False,False,False,False,False,False,False,False,False,False
2,W5226514,1249900,Brampton,Fletcher's Creek South,208 ROLLINGWOOD DRBrampton,L6Y5J6,"Location-Location- Border Of Mississauga, 4 Be...",House,Freehold,4,3,4,0,[AttachedGarage],8,42 x 110 FT ; As Per Survey,2,N/A (Finished),Detached,NotListed,NotListed,NotListed,Central air conditioning,Forced air (Natural gas),Brick,NotListed,Public Transit,True,False,False,False,False,False,False,False,False,False,False,False
3,W5234770,1249900,Brampton,Fletcher's West,92 MOFFATT AVEBrampton,L6Y4K8,Premium Corner Lot(4 Bdrms+2 Bdrms Finished Le...,House,Freehold,4,2,4,0,[AttachedGarage],4,51.76 x 97.83 FT ; Legal Basement,2,N/A (Finished),Detached,NotListed,NotListed,NotListed,Central air conditioning,Forced air (Natural gas),Brick,NotListed,"Public Transit, Schools",True,False,False,False,False,False,False,False,False,False,False,False
4,W5234047,1249900,Brampton,Fletcher's West,85 WHITE TAIL CRESBrampton,L6Y5C1,Beautiful Detached House With 4 Bedrooms And 4...,House,Freehold,4,0,4,0,[AttachedGarage],4,31.69 x 100 FT,2.5000,Full,Detached,NotListed,NotListed,NotListed,Central air conditioning,Forced air (Natural gas),Brick,NotListed,NotListed,True,False,False,False,False,False,False,False,False,False,False,False


## If it is true for Attached Garage or Detached Garage, the Garage value should be true too

In [12]:
df_3.loc[(df_3['Garage']==True)|(df_3['DetachedGarage']==True)|(df_3['AttachedGarage']==True), 'Garage'] = True


## totalParkingSpaces

In [13]:
df_3["totalParkingSpaces"].value_counts()

1     3267
2     1605
      ... 
63       1
19       1
Name: totalParkingSpaces, Length: 37, dtype: int64

In [14]:
df_3[df_3["typeBuilding"]=="Vacant Land"].totalParkingSpaces.value_counts()

NotListed    307
0             45
            ... 
10             1
5              1
Name: totalParkingSpaces, Length: 8, dtype: int64

## Assume for Vacant Land, if the # parking is not listed, it is 0.

In [15]:
df_3.loc[(df_3['typeBuilding']=="Vacant Land")&(df_3['totalParkingSpaces']=="NotListed"), 'totalParkingSpaces'] = '0'
df_3.loc[df_3['totalParkingSpaces']=="0", 'totalParkingSpaces'] = 0


In [16]:
df_3.columns

Index(['MLS', 'price', 'city', 'communityName', 'address', 'postal',
       'description', 'typeBuilding', 'title', 'bedroomAboveGrade',
       'bedroomBelowGrade', 'bathroomTotal', 'bathroomPartial', 'parkingType',
       'totalParkingSpaces', 'landSize', 'storeys', 'basementType', 'style',
       'buildingAmenities', 'maintenanceFees', 'maintenanceManagementCompany',
       'cooling', 'heatingType', 'exteriorFinish', 'buildingFeatures',
       'amenitiesNearby', 'AttachedGarage', 'Underground', 'InsideEntry',
       'Surfaced', 'Oversize', 'Gravel', 'Garage', 'Shared', 'DetachedGarage',
       'Carport', 'Interlocked', 'VisitorParking'],
      dtype='object')

## Land Size

In [17]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)
df_3["landSize"].value_counts()


NotListed                        4408
under 1/2 acre                    194
                                 ... 
37.5 x 100 FT ; Irregular Lot       1
22.97 x 130.19 FT                   1
Name: landSize, Length: 4698, dtype: int64

## Square Feet

In [18]:
df_3["landsizeSQFT"]=df_3["landSize"][df_3["landSize"].str.count(r'(.*[Xx].*[fF][tT])')==1]

In [19]:
df_3["landsizeSQFT_width"]=df_3["landsizeSQFT"].str.extract(r'([0-9.]+)')

In [20]:
df_3["landsizeSQFT_length"]=df_3["landsizeSQFT"].str.extract(r'([0-9]*[Xx].[0-9.]*)')
df_3["landsizeSQFT_length"]=df_3["landsizeSQFT_length"].str.replace("x", "")
df_3["landsizeSQFT_length"]=df_3["landsizeSQFT_length"].str.replace("X", "")


In [21]:
## get the square feet of land size
df_3["landsizeSQFT_width"]= pd.to_numeric(df_3["landsizeSQFT_width"], errors='coerce')
df_3["landsizeSQFT_length"]=pd.to_numeric(df_3["landsizeSQFT_length"], errors='coerce')
df_3["landsize_SQFT_result"]= df_3["landsizeSQFT_width"] * df_3["landsizeSQFT_length"]

## Square Meter

In [22]:
df_3["landsizeSQM"]=df_3["landSize"][df_3["landSize"].str.count(r'(.*[Xx].[0-9.]*\sM)')==1]
df_3["landsizeSQM_width"]=df_3["landsizeSQM"].str.extract(r'([0-9.]+)')
df_3["landsizeSQM_length"]=df_3["landsizeSQM"].str.extract(r'([0-9]*[Xx].[0-9.]*)')
df_3["landsizeSQM_length"]=df_3["landsizeSQM_length"].str.replace("x", "")
df_3["landsizeSQM_length"]=df_3["landsizeSQM_length"].str.replace("X", "")

df_3["landsizeSQM_width"]= pd.to_numeric(df_3["landsizeSQM_width"], errors='coerce')
df_3["landsizeSQM_length"]=pd.to_numeric(df_3["landsizeSQM_length"], errors='coerce')

df_3["landsizeSQM_resultSQFT"]=df_3["landsizeSQM_width"] * df_3["landsizeSQM_length"] * 10.764

df_3.loc[df_3['landsize_SQFT_result'].isnull(), 'landsize_SQFT_result'] = df_3['landsizeSQM_resultSQFT']


## acres

In [23]:
df_3["landsizeACRES"]=df_3["landSize"][df_3["landSize"].str.count(r'([0-9.]*acre)')==1]
df_3["landsizeACRES"]=df_3["landsizeACRES"].str.replace("1/2", "0.5")
df_3["landsizeACRES_result"]=df_3["landsizeACRES"].str[::-1].str.extract(r'(\d+.?\d+)', expand=False)
df_3["landsizeACRES_result"]=df_3["landsizeACRES_result"].str[::-1]

df_3["landsizeACRES_result"]= pd.to_numeric(df_3["landsizeACRES_result"], errors='coerce')

df_3["landsizeACRES_resultSQFT"] = df_3["landsizeACRES_result"] * 43560 ##convert acre to square foot

df_3.loc[df_3['landsize_SQFT_result'].isnull(), 'landsize_SQFT_result'] = df_3['landsizeACRES_resultSQFT']


## ac

In [24]:
df_3["landsizeAC"]=df_3["landSize"][df_3["landSize"].str.count(r'([0-9.]*\sac$)')==1]
df_3["landsizeAC_result"]=df_3["landsizeAC"].str.replace(' ac','')
df_3["landsizeAC_result"]= pd.to_numeric(df_3["landsizeAC_result"], errors='coerce')
df_3["landsizeAC_resultSQFT"] = df_3["landsizeAC_result"] * 43560 ##convert acre to square foot

df_3.loc[df_3['landsize_SQFT_result'].isnull(), 'landsize_SQFT_result'] = df_3['landsizeAC_resultSQFT']


In [25]:
## remove "landsizeSQFT_width" ,"landsizeSQFT_length" and "landsizeSQFT"
## rename "landsize_SQFT_result" to "landSize(sqft)"
df_3=df_3.drop(["landsizeSQFT_width", "landsizeSQFT_length", "landsizeSQFT", "landsizeSQM_width", "landsizeSQM_length", "landsizeSQM", "landsizeSQM_resultSQFT", "landsizeACRES","landsizeACRES_result", "landsizeACRES_resultSQFT", "landsizeAC", "landsizeAC_result", "landsizeAC_resultSQFT"], axis=1)
df_3 = df_3.rename(columns={'landsize_SQFT_result':'landSize_sqft'})

In [26]:
df_3.iloc[[4586]]

,MLS,price,city,communityName,address,postal,description,typeBuilding,title,bedroomAboveGrade,bedroomBelowGrade,bathroomTotal,bathroomPartial,parkingType,totalParkingSpaces,landSize,storeys,basementType,style,buildingAmenities,maintenanceFees,maintenanceManagementCompany,cooling,heatingType,exteriorFinish,buildingFeatures,amenitiesNearby,AttachedGarage,Underground,InsideEntry,Surfaced,Oversize,Gravel,Garage,Shared,DetachedGarage,Carport,Interlocked,VisitorParking,landSize_sqft
4586,1236006,1150000,Ottawa,NotListed,146 RIVINGTON STREETCarp,K0A1L0,DEVELOPMENT OPPOURTUNITY! Waterfront ideal sit...,Vacant Land,Freehold,0,0,0,0,[NotListed],0,0.76 ac,NotListed,Unknown (Not Applicable),NotListed,NotListed,NotListed,NotListed,NotListed,NotListed,NotListed,NotListed,"Shopping, Water Nearby",False,False,False,False,False,False,False,False,False,False,False,False,33105.6


In [27]:
#missing values
df_3.landSize_sqft.isna().sum()

4570

## Can be deleted later, START

## Check the landSize value other than " x FT"

## Get the square Meter, and the specific acre. 
## under 1/2 acre ==1/2 acre
## Others are considered missing.

In [28]:
series=df_3["landSize"][df_3["landSize"].str.count(r'(.*[Xx].*[fF][tT])')!=1]
series

1              NotListed
15       45.11 x 82.82 M
              ...       
10248          NotListed
10250          NotListed
Name: landSize, Length: 5685, dtype: object

In [29]:
series=df_3["landSize"][df_3["landSize"].str.count(r'(.*[Xx].[0-9.]*\sM)')==1]
series

15       45.11 x 82.82 M
26        15.25 x 30.5 M
              ...       
9979      4.35 x 12.46 M
10159           4 x 17 M
Name: landSize, Length: 26, dtype: object

In [30]:
series=df_3["landSize"][df_3["landSize"].str.count(r'([0-9.]*acre)')==1]
series

58      under 1/2 acre
65      under 1/2 acre
             ...      
9855    under 1/2 acre
9898    under 1/2 acre
Name: landSize, Length: 910, dtype: object

In [31]:
series=series.str.replace("1/2", "0.5")
new=series.str[::-1].str.extract(r'(\d+.?[\d+]?)', expand=False)
#(r'((?=.*)\s.*(?=.ac))')#r'(\s.*(?=.ac))')

new=new.str[::-1]
new

58      0.5
65      0.5
       ... 
9855    0.5
9898    0.5
Name: landSize, Length: 910, dtype: object

In [32]:
seriesnew=series.str.extract(r'([0-9]*[Xx].[0-9.]*)')
seriesnew[0]=seriesnew[0].str.replace("x", "")
seriesnew[0]=seriesnew[0].str.replace("X", "")
seriesnew

,0
58,NaN
65,NaN
...,...
9855,NaN
9898,NaN


In [33]:
series=df_3["landSize"][df_3["landSize"].str.count(r'([0-9.]*acre)')==1]
series

58      under 1/2 acre
65      under 1/2 acre
             ...      
9855    under 1/2 acre
9898    under 1/2 acre
Name: landSize, Length: 910, dtype: object

In [34]:
series=df_3["landSize"][df_3["landSize"].str.count(r'([0-9.]*\sac$)')==1]
series

3452     1.5 ac
3453    0.18 ac
         ...   
4780     0.2 ac
4791    2.85 ac
Name: landSize, Length: 288, dtype: object

In [35]:
new=series.str.replace(' ac','')
#(r'((?=.*)\s.*(?=.ac))')#r'(\s.*(?=.ac))')
#new=series.str.extract(r'(\d+.?[\d+]?)', expand=False)
new

3452     1.5
3453    0.18
        ... 
4780     0.2
4791    2.85
Name: landSize, Length: 288, dtype: object

## Can be deleted, END

In [36]:
#pd.set_option("display.max_rows", 5)

## Deal with storeys

In [37]:
df_3["storeys"].value_counts()

NotListed    4143
2            3570
             ... 
0               1
24              1
Name: storeys, Length: 26, dtype: int64

In [38]:
df_3[df_3["storeys"]=="NotListed"].typeBuilding.value_counts()

Apartment              3148
House                   401
                       ... 
Other                     2
Manufactured/mobile       1
Name: typeBuilding, Length: 9, dtype: int64

## For Vacant Land, Assume there is no storey

In [39]:
df_3.loc[(df_3['storeys']=="NotListed")&(df_3['typeBuilding']=="Vacant Land"), "storeys"] = 0
df_3.loc[df_3['storeys']=="0", 'storeys'] = 0


## For Not Listed, all building types except vacant land , assume there is only 1 storey

In [40]:
df_3.loc[(df_3['storeys']=="NotListed")&(df_3['typeBuilding']!="Vacant Land"), "storeys"] = 1
df_3.loc[df_3['storeys']=="1", 'storeys'] = 1


## Basement Type

In [41]:
df_3["basementType"].value_counts()

NotListed                       3777
N/A (Finished)                  2580
                                ... 
Unknown (Partially finished)       1
Partial (Not Applicable)           1
Name: basementType, Length: 35, dtype: int64

The basement types I developed as follows:  
  * None
  * Full
  * Partial
  * Cellar
  * Craw Space
  * Common  
The basement finish as follows:
  * Finished
  * Partially Finished
  * Unfinished
  * None

## Add new column basementTypeN and  basementFinish

## Assume that  properties whose basement are not listed has no basement

In [42]:
df_3.loc[df_3['basementType']=="NotListed", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="NotListed", "basementFinish"] = "None"

## Process basementType "None"

In [43]:
df_3.loc[df_3['basementType']=="N/A (Finished)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="N/A (Finished)", "basementFinish"] = "Finished"

In [44]:
df_3.loc[df_3['basementType']=="None (Not Applicable)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="None (Not Applicable)", "basementFinish"] = "None"

In [45]:
df_3.loc[df_3['basementType']=="N/A (Unfinished)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="N/A (Unfinished)", "basementFinish"] = "Unfinished"

In [46]:
df_3.loc[df_3['basementType']=="N/A (Partially finished)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="N/A (Partially finished)", "basementFinish"] = "Partially finished"

In [47]:
df_3.loc[df_3['basementType']=="Unknown (Not Applicable)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="Unknown (Not Applicable)", "basementFinish"] = "None"

In [48]:
df_3.loc[df_3['basementType']=="Unknown (Unfinished)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="Unknown (Unfinished)", "basementFinish"] = "Unfinished"

In [49]:
df_3.loc[df_3['basementType']=="None (Unfinished)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="None (Unfinished)", "basementFinish"] = "Unfinished"

In [50]:
df_3.loc[df_3['basementType']=="See Remarks (Not Applicable)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="See Remarks (Not Applicable)", "basementFinish"] = "None"

In [51]:
df_3.loc[df_3['basementType']=="None (Finished)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="None (Finished)", "basementFinish"] = "Finished"

In [52]:
df_3.loc[df_3['basementType']=="Unknown (Finished)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="Unknown (Finished)", "basementFinish"] = "Finished"

In [53]:
df_3.loc[df_3['basementType']=="Unknown (Partially finished)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="Unknown (Partially finished)", "basementFinish"] = "Partially finished"

In [54]:
df_3.loc[df_3['basementType']=="See Remarks (Finished)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="See Remarks (Finished)", "basementFinish"] = "Finished"


In [55]:
df_3.loc[df_3['basementType']=="See Remarks (Unfinished)", "basementTypeN"] = "None"
df_3.loc[df_3['basementType']=="See Remarks (Unfinished)", "basementFinish"] = "Unfinished"

## Process basementType Full

In [56]:
df_3.loc[df_3['basementType']=="Full (Finished)", "basementTypeN"] = "Full"
df_3.loc[df_3['basementType']=="Full (Finished)", "basementFinish"] = "Finished"

In [57]:
df_3.loc[df_3['basementType']=="Full (Unfinished)", "basementTypeN"] = "Full"
df_3.loc[df_3['basementType']=="Full (Unfinished)", "basementFinish"] = "Unfinished"

In [58]:
df_3.loc[df_3['basementType']=="Full (Partially finished)", "basementTypeN"] = "Full"
df_3.loc[df_3['basementType']=="Full (Partially finished)", "basementFinish"] = "Partially finished"

 Assume "Full" basement type is unfinished

In [59]:
df_3.loc[df_3['basementType']=="Full", "basementTypeN"] = "Full"
df_3.loc[df_3['basementType']=="Full", "basementFinish"] = "Unfinished"

In [60]:
df_3.loc[df_3['basementType']=="Full (Not Applicable)", "basementTypeN"] = "Full"
df_3.loc[df_3['basementType']=="Full (Not Applicable)", "basementFinish"] = "Unfinished"


## Process basementType Partial

In [61]:
df_3.loc[df_3['basementType']=="Partial (Unfinished)", "basementTypeN"] = "Partial"
df_3.loc[df_3['basementType']=="Partial (Unfinished)", "basementFinish"] = "Unfinished"

In [62]:
df_3.loc[df_3['basementType']=="Partial (Finished)", "basementTypeN"] = "Partial"
df_3.loc[df_3['basementType']=="Partial (Finished)", "basementFinish"] = "Finished"

assume "partial" is unfinished

In [63]:
df_3.loc[df_3['basementType']=="Partial", "basementTypeN"] = "Partial"
df_3.loc[df_3['basementType']=="Partial", "basementFinish"] = "Unfinished"

In [64]:
df_3.loc[df_3['basementType']=="Partial (Not Applicable)", "basementTypeN"] = "Partial"
df_3.loc[df_3['basementType']=="Partial (Not Applicable)", "basementFinish"] = "Unfinished"

In [65]:
df_3.loc[df_3['basementType']=="Partial (Partially finished)", "basementTypeN"] = "Partial"
df_3.loc[df_3['basementType']=="Partial (Partially finished)", "basementFinish"] = "Partially finished"


## Process basementType Cellar

In [66]:
df_3.loc[df_3['basementType']=="Cellar (Unfinished)", "basementTypeN"] = "Cellar"
df_3.loc[df_3['basementType']=="Cellar (Unfinished)", "basementFinish"] = "Unfinished"

## Process basementType Craw Space

Assume "Crawl Space" is unfinished

In [67]:
df_3.loc[df_3['basementType']=="Crawl space", "basementTypeN"] = "Crawl Space"
df_3.loc[df_3['basementType']=="Crawl space", "basementFinish"] = "Unfinished"


In [68]:
df_3.loc[df_3['basementType']=="Crawl space (Unfinished)", "basementTypeN"] = "Crawl Space"
df_3.loc[df_3['basementType']=="Crawl space (Unfinished)", "basementFinish"] = "Unfinished"

In [69]:
df_3.loc[df_3['basementType']=="Crawl space (Finished)", "basementTypeN"] = "Crawl Space"
df_3.loc[df_3['basementType']=="Crawl space (Finished)", "basementFinish"] = "Finished"

In [70]:
df_3.loc[df_3['basementType']=="Crawl space (Partially finished)", "basementTypeN"] = "Crawl Space"
df_3.loc[df_3['basementType']=="Crawl space (Partially finished)", "basementFinish"] = "Partially finished"

Assume "Crawl Space (Not Applicable)" is unfinished

In [71]:
df_3.loc[df_3['basementType']=="Crawl space (Not Applicable)", "basementTypeN"] = "Crawl Space"
df_3.loc[df_3['basementType']=="Crawl space (Not Applicable)", "basementFinish"] = "Unfinished"

## process basementType Common

Assume "Common (Not Applicable)" is unfinished

In [72]:
df_3.loc[df_3['basementType']=="Common (Not Applicable)", "basementTypeN"] = "Common"
df_3.loc[df_3['basementType']=="Common (Not Applicable)", "basementFinish"] = "Unfinished"

In [73]:
df_3.loc[df_3['basementType']=="Common (Unfinished)", "basementTypeN"] = "Common"
df_3.loc[df_3['basementType']=="Common (Unfinished)", "basementFinish"] = "Unfinished"


In [74]:
df_3.loc[df_3['basementType']=="Common (Finished)", "basementTypeN"] = "Common"
df_3.loc[df_3['basementType']=="Common (Finished)", "basementFinish"] = "Finished"

In [75]:
df_3.loc[df_3['basementType']=="Common (Partially finished)", "basementTypeN"] = "Common"
df_3.loc[df_3['basementType']=="Common (Partially finished)", "basementFinish"] = "Partially finished"

In [76]:
df_3["basementTypeN"].value_counts()

None      7213
Full      2056
          ... 
Common      35
Cellar       4
Name: basementTypeN, Length: 6, dtype: int64

In [77]:
df_3["basementFinish"].value_counts()

None                  4182
Finished              3744
Unfinished            1079
Partially finished     454
Name: basementFinish, dtype: int64

## Deal with "style" column

In [78]:
df_3["style"].value_counts()

NotListed       5019
Detached        3773
                ... 
Link              10
Side by side       2
Name: style, Length: 8, dtype: int64

In [79]:
df_3[df_3["style"]=="NotListed"].typeBuilding.value_counts()

Apartment          3583
Row / Townhouse     973
                   ... 
Modular               3
Mixed                 2
Name: typeBuilding, Length: 10, dtype: int64

In [80]:
df_3[df_3["style"]=="Detached"].typeBuilding.value_counts()

House                  3768
Manufactured/mobile       3
Apartment                 1
Vacant Land               1
Name: typeBuilding, dtype: int64

In [81]:
df_3[df_3["style"]=="Semi-detached"].typeBuilding.value_counts()

House     768
Duplex      1
Name: typeBuilding, dtype: int64

In [82]:
df_3[df_3["style"]=="Attached"].typeBuilding.value_counts()

Row / Townhouse    541
Apartment           71
Name: typeBuilding, dtype: int64

In [83]:
df_3[df_3["typeBuilding"]=="Duplex"]["style"].value_counts()

NotListed        39
Up and down       4
Semi-detached     1
Side by side      1
Name: style, dtype: int64

In [84]:
df_3[df_3["typeBuilding"]=="Row / Townhouse"]["style"].value_counts()

NotListed    973
Attached     541
Name: style, dtype: int64

In [85]:
df_3[df_3["typeBuilding"]=="Vacant Land"]["style"].value_counts()

NotListed    362
Detached       1
Name: style, dtype: int64

## for apartment, if not listed, then use value "Attached"

In [86]:
df_3.loc[(df_3['typeBuilding']=="Apartment")&(df_3['style']=="NotListed"), "style"] = "Attached"

## for townhouse, if not listed, use "Attached"

In [87]:
df_3.loc[(df_3['typeBuilding']=="Row / Townhouse")&(df_3['style']=="NotListed"), "style"] = "Attached"

## for duplex, triplex and fourplex, if not listed, use "Attached"

In [88]:
df_3.loc[(df_3['typeBuilding']=="Duplex")&(df_3['style']=="NotListed"), "style"] = "Attached"

In [89]:
df_3.loc[(df_3['typeBuilding']=="Triplex")&(df_3['style']=="NotListed"), "style"] = "Attached"

In [90]:
df_3.loc[(df_3['typeBuilding']=="Fourplex")&(df_3['style']=="NotListed"), "style"] = "Attached"

## For vacant land, use value "Vacant Land"

In [91]:
df_3.loc[(df_3['typeBuilding']=="Vacant Land")&(df_3['style']=="NotListed"), "style"] = "Vacant Land"

## For House, if not listed, use "Detached"

In [92]:
df_3.loc[(df_3['typeBuilding']=="House")&(df_3['style']=="NotListed"), "style"] = "Detached"

## For modular, assume "detached"

In [93]:
df_3.loc[(df_3['typeBuilding']=="Modular")&(df_3['style']=="NotListed"), "style"] = "Detached"

## For Other typeBuilding, assume "Detached"

In [94]:
df_3.loc[(df_3['typeBuilding']=="Other")&(df_3['style']=="NotListed"), "style"] = "Detached"


## For Mixed, assume "Attached"

In [95]:
df_3.loc[(df_3['typeBuilding']=="Mixed")&(df_3['style']=="NotListed"), "style"] = "Attached"

In [96]:
df_3["style"].value_counts()

Attached        5253
Detached        3789
                ... 
Link              10
Side by side       2
Name: style, Length: 8, dtype: int64

In [97]:
df_3=df_3.rename(columns={"style": "styleAttach"})

## buildingAmenities

In [98]:
df_3["buildingAmenities"].value_counts()

NotListed                                                            6497
Storage - Locker, Security/Concierge, Party Room, Exercise Centre     622
                                                                     ... 
Storage - Locker, Sauna, Security/Concierge, Recreation Centre          1
Other, Party Room, Sauna, Laundry Facility                              1
Name: buildingAmenities, Length: 185, dtype: int64

## Convert buildingAmenities to multiple columns with true/false value

In [99]:
#split different tags
df_3['buildingAmenities'] = df_3['buildingAmenities'].str.replace(" ","")
df_3['buildingAmenities'] = df_3['buildingAmenities'].str.split(pat=',')

# collect all unique tags from those lists
tags = set(df_3['buildingAmenities'].explode().values)
print(tags)

{'Clubhouse', 'Other', 'CarWash', 'MusicRoom', 'Storage-Locker', 'Sauna', 'PartyRoom', 'RecreationCentre', 'GuestSuite', 'Furnished', 'LaundryFacility', 'NotListed', 'ExerciseCentre', 'Laundry-InSuite', 'Security/Concierge', 'Whirlpool'}


In [100]:
# create a new Boolean column for each tag
for tag in tags:
    df_3[tag] = [tag in df_3['buildingAmenities'].loc[i] for i in df_3.index]

In [101]:
amenitiesCol=['CarWash', 'ExerciseCentre', 'Other', 'Clubhouse', 'RecreationCentre', 'NotListed', 'Security/Concierge', 'Laundry-InSuite', 'Furnished', 'GuestSuite', 'MusicRoom', 'PartyRoom', 'LaundryFacility', 'Whirlpool', 'Sauna', 'Storage-Locker']

In [102]:
newdict={}
for col in amenitiesCol:
    newdict[col]=df_3[col][df_3[col]==True].count()
    #print(col+" "+str(df_3[col][df_3[col]==True].count()))
sorted_keys = sorted(newdict, key=newdict.get, reverse=True)  # [1, 3, 2]
sorted_dict = {}
for w in sorted_keys:
    sorted_dict[w] = newdict[w]

print(sorted_dict) # {1: 1, 3: 4, 2: 9}

{'NotListed': 6497, 'ExerciseCentre': 2746, 'Storage-Locker': 2439, 'Security/Concierge': 2158, 'PartyRoom': 1991, 'RecreationCentre': 625, 'Sauna': 504, 'Laundry-InSuite': 334, 'CarWash': 132, 'GuestSuite': 78, 'LaundryFacility': 48, 'Other': 11, 'Whirlpool': 10, 'Clubhouse': 4, 'Furnished': 4, 'MusicRoom': 2}


## Whirlpool is a kind of hot tub

## Remove column "NotListed", and "Other"

In [103]:
df_3 = df_3.drop(["NotListed", "Other"], axis=1)

## Output Data, and then go to Part 3 code 

In [105]:
df_3.to_csv(output_folder+"/"+"all_after_preprocessing2.csv", index=False)